In [2]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
import seaborn as sns
import re
import random as rd
import tensorflow as tf
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization
from nltk import word_tokenize, RegexpTokenizer
from nltk.corpus import stopwords, wordnet
from sklearn.preprocessing import OneHotEncoder
import matplotlib.pyplot as plt
import autokeras as ak
from sklearn.model_selection import KFold

import nltk
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\ddayv\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\ddayv\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [3]:
def Tokenize(f):     ## Pre-processando a frase

    ## Colocando em minusculo
    ## Retirando a pontuaçao
    ## Retirando as StopWords
    
    f = f.lower().replace('\n', '').replace('-','').replace('#','').replace('.','').replace(',','').replace('!','').replace('r\n','').replace('  ','').replace('https','').replace('rt','').replace('rn','')
    token = RegexpTokenizer(r"\w+")
    f = token.tokenize(f)
    
    stop_words = set(stopwords.words('portuguese'))
    
    new_word = [word for word in f if not word in stop_words]
    
    return ' '.join(new_word)

def remove_user(frase):
    frase = re.sub('@\w+','',frase)
    frase = re.sub('{https}[^ ]+','',frase)
    frase = re.sub('https\w+','',frase)
    # re.sub('#\w+','',frase)
    return frase

def pre_X(frases):
    lista = []
    
    for frase in frases:
        lista.append(frase)
        
    return lista

def pre_Y(number):
    lista = []
    
    for numb in number:
        lista.append(numb)
    
    return lista

def set_array(frases):
    
    vocab = []
    palavras = []
    for frase in frases:
        
        text_array = remove_user(frase)
        text_array = Tokenize(text_array)
        text_array = text_array.split(' ')
        for i in range(len(text_array)):
            vocab.append(text_array[i])
    
    
        
    return vocab


## TWEETSENT

In [12]:
df_tweet= pd.read_csv('export_TweetSentBR.csv')
df_tweet

,id,id_twitter,text,sentiment
0,1343,863044774588272640,Que coisa linda! O Programa #encontro estava m...,1
1,1344,865583716088766467,"Por mais #Encontro com as Irmãs Galvão, adorei...",1
2,1345,865063232201011201,Mr. CATRA @OficialMrCatra lançando sua nova mú...,1
3,1346,864668391008763905,quem viu aquela lutadora modela barbuda tatuad...,0
4,1347,865572794016378882,Tô passada com esse cara.... quanta merda pode...,-1
...,...,...,...,...
11565,12908,864636619000877056,eu ja to aqui pronto pro #MasterChefBR mas ain...,-1
11566,12909,863581588713603072,MALUCO! Uma coisa que eu não tenho coragem é e...,-1
11567,12910,864831041349054464,#MaisVoce @ANAMARIABRAGA está linda @RedeGlobo,1
11568,12911,863042798575951872,"Que orgulho de ti, @sportrecife! #Encontro",1


In [13]:
df_remove = df_tweet[df_tweet['sentiment'] == '-']
df_tweet = df_tweet.drop(df_remove.index)

df_tweet['text'] = df_tweet['text'].apply(remove_user)
df_tweet['text'] = df_tweet['text'].apply(Tokenize)

In [14]:
df_tweet['sentiment'] = df_tweet['sentiment'].apply(lambda x: int(x))

In [15]:
text_Tweetsent = df_tweet['text']
polarity = np.asarray(df_tweet['sentiment'])

In [16]:
X_text = count_vect.transform(text_Tweetsent)

X_text_transform = tfidf_transformer.transform(X_text) # Aplicando o TF-IDF
X_text_transform.shape

(11533, 25852)

In [17]:
X_train1, X_test_TWEET, Y_train1, Y_test_TWEET = train_test_split(X_text_transform, polarity, test_size=0.3)

In [18]:
X_train12, X_test_TWEET1, Y_train12, Y_test_TWEET1 = train_test_split(df_tweet['text'], df_tweet['sentiment'] , test_size=0.3)

## TASH

In [19]:
df = pd.read_csv("tash-pt.csv")
df = df.dropna()
df = df.drop(columns=['id_twitter'])


In [20]:
df['text'] = df['text'].apply(remove_user)
df['text'] = df['text'].apply(Tokenize)

In [21]:
text_TASH = df['text']
sentiment = np.asarray(df['sentiment'])

X_text_TASH = count_vect.transform(text_TASH)
X_text_TASH_ = tfidf_transformer.transform(X_text_TASH) # Aplicando o TF-IDF
X_text_TASH_.shape

(2787, 25852)

In [22]:
X_train2, X_test_TASH, Y_train2, Y_test_TASH = train_test_split(X_text_TASH_, sentiment, test_size=0.3)

In [23]:
X_train21, X_test_TASH21, Y_train21, Y_test_TASH21 = train_test_split(df['text'], np.asarray(df['sentiment']), test_size=0.3)

## KANSOAN

In [24]:
df = pd.read_csv("data.txt", sep=',', header=None, names=['text','sentiment'])

df_remove = df[df['sentiment'] == '#Inveja'].index
df = df.drop(df_remove)

df_remove = df[df['sentiment'] == '#Raiva'].index
df = df.drop(df_remove)

df_remove = df[df['sentiment'] == '#Ironia'].index
df = df.drop(df_remove)


def binario(termo):
    if termo == '#Feliz' or termo == '#Amor':
        return 1
    elif termo == '#Triste' or termo == '#Chateado':
        return 0       
    
df['sentiment'] = df['sentiment'].apply(binario)

In [25]:
df['text'] = df['text'].apply(remove_user)
df['text'] = df['text'].apply(Tokenize)

In [26]:
text_KANSOAN = df['text']
sentiment = np.asarray(df['sentiment'])

X_text_KANSOAN = count_vect.transform(text_KANSOAN)
X_text_KANSOAN_ = tfidf_transformer.transform(X_text_KANSOAN) # Aplicando o TF-IDF
X_text_KANSOAN_.shape

(9224, 25852)

In [27]:
X_train3, X_test_KANSOAN, Y_train3, Y_test_KANSOAN = train_test_split(X_text_KANSOAN_, sentiment, test_size=0.3)

In [28]:
X_train31, X_test_KANSOAN1, Y_train31, Y_test_KANSOAN1 = train_test_split(df['text'], np.asarray(df['sentiment']), test_size=0.3)

## TEST

In [4]:
df = pd.read_excel('./rotulação/TweetsPolitical01 OK OK.xlsx')
df2 = pd.read_excel('./rotulação/TweetsPolitical02 OK OK.xlsx')
df3 = pd.read_excel('./rotulação/TweetsPolitical03 OK OK.xlsx')
df4 = pd.read_excel('./rotulação/TweetsPolitical04 OK OK.xlsx')
df5 = pd.read_excel('./rotulação/TweetsPolitical05 OK OK.xlsx')
df6 = pd.read_excel('./rotulação/TweetsPolitical06 OK OK.xlsx')
df7 = pd.read_excel('./rotulação/TweetsPolitical07 OK OK.xlsx')
df8 = pd.read_excel('./rotulação/TweetsPolitical08 OK OK.xlsx')
df9 = pd.read_excel('./rotulação/TweetsPolitical09 OK OK.xlsx')
df10 = pd.read_excel('./rotulação/TweetsPolitical10 OK OK.xlsx')
df11 = pd.read_excel('./rotulação/TweetsPolitical11 OK OK.xlsx')
df12 = pd.read_excel('./rotulação/TweetsPolitical12 OK OK.xlsx')
df13 = pd.read_excel('./rotulação/TweetsPolitical13 OK OK.xlsx')
df14 = pd.read_excel('./rotulação/TweetsPolitical14 OK OK.xlsx')
df15 = pd.read_excel('./rotulação/TweetsPolitical15 OK OK.xlsx')

lista = [df,df2,df3,df4,df5,df6,df7,df8,df9,df10,df11,df12,df13,df14,df15]

df_all = pd.concat(lista)
df_all.shape

(12670, 2)

In [5]:
df_all = df_all.dropna()

In [6]:
df_all.drop(df_all[df_all['Polaridade'] == 11].index, inplace=True)
df_all.drop(df_all[df_all['Polaridade'] == -2].index, inplace=True)
df_all.drop(df_all[df_all['Polaridade'] == 10].index, inplace=True)

df_all['Polaridade'].unique()

array([ 0.,  1., -1.])

In [7]:
df_all.head()

,Tweet,Polaridade
0,- #CaoNossoDeCadaDia #Novo - Vanessa Mandotti ...,0.0
1,"- Bola pra frente que amanhã é outro dia, outr...",1.0
2,- Cara de mal? Acho que não... Apenas um corte...,1.0
3,""" #PCdoB O PARTIDO QUE FAZ A DIFERENÇA: LEAL E...",1.0
4,"""@camaradamae: #PCdoB o Partido da coragem ! ...",1.0


In [8]:
df_all['Tweet'] = df_all['Tweet'].apply(remove_user)
df_all['Tweet'] = df_all['Tweet'].apply(Tokenize)
polarity = np.asarray(df_all['Polaridade'])
Tweet = df_all['Tweet']


In [9]:
count_vect = CountVectorizer()

Tweet = count_vect.fit_transform(Tweet)

tfidf_transformer = TfidfTransformer()
X_train_transform = tfidf_transformer.fit_transform(Tweet) # Aplicando o TF-IDF
X_train_transform.shape

(12626, 25852)

In [10]:
X_train, X_test, Y_train, Y_test = train_test_split(X_train_transform, polarity, test_size=0.3)



In [11]:
X_train_text, X_test_text, Y_train_text, Y_test_text = train_test_split(df_all['Tweet'] , df_all['Polaridade'], test_size=0.3)


## NB - TRAIN -> TWEETSENT- TEST -> UNILEX

In [29]:
clf = MultinomialNB()
clf.fit(X_train1, Y_train1)
y_pred = clf.predict(X_test)

print(classification_report(Y_test,y_pred))

              precision    recall  f1-score   support

        -1.0       0.58      0.29      0.38      1284
         0.0       0.37      0.08      0.13      1373
         1.0       0.32      0.81      0.46      1131

    accuracy                           0.37      3788
   macro avg       0.42      0.39      0.32      3788
weighted avg       0.43      0.37      0.31      3788



## NB - TRAIN -> TWEETSENT - TEST -> TASH

In [30]:
clf = MultinomialNB()
clf.fit(X_train1, Y_train1)
y_pred = clf.predict(X_test_TASH)

print(classification_report(Y_test_TASH,y_pred))

              precision    recall  f1-score   support

          -1       0.43      0.34      0.38       238
           0       0.44      0.03      0.06       315
           1       0.38      0.83      0.52       284

    accuracy                           0.39       837
   macro avg       0.42      0.40      0.32       837
weighted avg       0.42      0.39      0.31       837



## NB - TRAIN -> TWEET - TEST -> KANSOAN

In [31]:
def func(num):
    if num == 0:
        return -1
    else:
        return 1

sentiment = df['sentiment'].apply(func)
X_train3, X_test_KANSOAN, Y_train3, Y_test_KANSOAN = train_test_split(X_text_KANSOAN_, sentiment, test_size=0.3)

In [32]:
clf = MultinomialNB()
clf.fit(X_train1, Y_train1)
y_pred = clf.predict(X_test_KANSOAN)

print(classification_report(Y_test_KANSOAN,y_pred, zero_division=True))

              precision    recall  f1-score   support

          -1       0.97      0.62      0.75      1352
           0       0.00      1.00      0.00         0
           1       0.73      0.98      0.84      1416

    accuracy                           0.80      2768
   macro avg       0.57      0.86      0.53      2768
weighted avg       0.85      0.80      0.80      2768



## SVM - TRAIN->UNILEX - TEST->TWEETSENT

In [33]:
clf = SVC().fit(X_train1, Y_train1) 
y_pred = clf.predict(X_test_TWEET)

print(classification_report(Y_test_TWEET,y_pred))

              precision    recall  f1-score   support

          -1       0.60      0.53      0.56      1010
           0       0.51      0.28      0.36       850
           1       0.62      0.81      0.70      1600

    accuracy                           0.60      3460
   macro avg       0.58      0.54      0.54      3460
weighted avg       0.59      0.60      0.58      3460



## SVM T TWEET T TASH

In [34]:
clf = SVC().fit(X_train1, Y_train1) 
y_pred = clf.predict(X_test_TASH)

print(classification_report(Y_test_TASH,y_pred))

              precision    recall  f1-score   support

          -1       0.42      0.48      0.45       238
           0       0.42      0.17      0.25       315
           1       0.41      0.63      0.49       284

    accuracy                           0.41       837
   macro avg       0.42      0.43      0.40       837
weighted avg       0.42      0.41      0.39       837



## SVM T TWEET T KANSOAN

In [35]:
clf = SVC().fit(X_train1, Y_train1) 
y_pred = clf.predict(X_test_KANSOAN)

print(classification_report(Y_test_KANSOAN,y_pred, zero_division=True))

              precision    recall  f1-score   support

          -1       0.94      0.63      0.75      1352
           0       0.00      1.00      0.00         0
           1       0.78      0.92      0.85      1416

    accuracy                           0.78      2768
   macro avg       0.57      0.85      0.53      2768
weighted avg       0.86      0.78      0.80      2768



## RL T TWEETSENT T UNILEX

In [36]:
clf = LogisticRegression(max_iter=1000).fit(X_train1, Y_train1) 
y_pred = clf.predict(X_test)

print(classification_report(Y_test,y_pred))

              precision    recall  f1-score   support

        -1.0       0.56      0.24      0.34      1284
         0.0       0.37      0.36      0.37      1373
         1.0       0.33      0.56      0.42      1131

    accuracy                           0.38      3788
   macro avg       0.42      0.39      0.38      3788
weighted avg       0.43      0.38      0.37      3788



## RL T TWEET T TASH

In [37]:
clf = LogisticRegression(max_iter=1000).fit(X_train1, Y_train1) 
y_pred = clf.predict(X_test_TASH)

print(classification_report(Y_test_TASH,y_pred))

              precision    recall  f1-score   support

          -1       0.41      0.50      0.45       238
           0       0.47      0.30      0.36       315
           1       0.45      0.54      0.49       284

    accuracy                           0.44       837
   macro avg       0.44      0.45      0.44       837
weighted avg       0.44      0.44      0.43       837



## RL T TWEET T KANSOAN

In [38]:
clf = LogisticRegression(max_iter=1000).fit(X_train1, Y_train1) 
y_pred = clf.predict(X_test_KANSOAN)

print(classification_report(Y_test_KANSOAN,y_pred, zero_division=True))

              precision    recall  f1-score   support

          -1       0.94      0.71      0.81      1352
           0       0.00      1.00      0.00         0
           1       0.86      0.89      0.87      1416

    accuracy                           0.80      2768
   macro avg       0.60      0.87      0.56      2768
weighted avg       0.90      0.80      0.84      2768



## TOTALMENT.C T TWEETSENT T UNILEX

In [39]:
model = tf.keras.Sequential([
    tf.keras.layers.Dense(50, activation='relu'),
    tf.keras.layers.Dropout(0.3),
    tf.keras.layers.Dense(25, activation='relu'),
    tf.keras.layers.Dropout(0.3),
    tf.keras.layers.Dense(10, activation='tanh'),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(3 , activation='softmax')
])

model.compile(
    optimizer='adam',
    loss=tf.keras.losses.categorical_crossentropy,
    metrics=['accuracy']
)

one = OneHotEncoder(sparse=False)

y_one = one.fit_transform(np.asarray(Y_train1).reshape(-1,1))
y_one_test = one.fit_transform(Y_test.reshape(-1,1))

fit = model.fit(X_train1.todense(), y_one, epochs=10, validation_data=(X_test.todense(), y_one_test))

Epoch 1/10
253/253 [==============================] - 3s 12ms/step - loss: 1.0036 - accuracy: 0.4915 - val_loss: 1.0799 - val_accuracy: 0.3936
Epoch 2/10
253/253 [==============================] - 2s 9ms/step - loss: 0.8295 - accuracy: 0.6182 - val_loss: 1.0738 - val_accuracy: 0.4253
Epoch 3/10
253/253 [==============================] - 2s 10ms/step - loss: 0.7009 - accuracy: 0.7067 - val_loss: 1.1208 - val_accuracy: 0.4303
Epoch 4/10
253/253 [==============================] - 2s 8ms/step - loss: 0.5771 - accuracy: 0.7749 - val_loss: 1.2113 - val_accuracy: 0.4377
Epoch 5/10
253/253 [==============================] - 2s 9ms/step - loss: 0.4815 - accuracy: 0.8205 - val_loss: 1.3183 - val_accuracy: 0.4372
Epoch 6/10
253/253 [==============================] - 2s 9ms/step - loss: 0.4051 - accuracy: 0.8445 - val_loss: 1.4754 - val_accuracy: 0.4298
Epoch 7/10
253/253 [==============================] - 2s 8ms/step - loss: 0.3510 - accuracy: 0.8693 - val_loss: 1.5378 - val_accuracy: 0.4258
Epoc

In [40]:
predicted = model.predict(X_test.todense())

print(classification_report(np.argmax(y_one_test, axis=1), np.argmax(predicted, axis=1)))

              precision    recall  f1-score   support

           0       0.47      0.43      0.45      1284
           1       0.41      0.56      0.47      1373
           2       0.41      0.27      0.33      1131

    accuracy                           0.43      3788
   macro avg       0.43      0.42      0.42      3788
weighted avg       0.43      0.43      0.42      3788



## TASH

In [41]:
predicted = model.predict(X_test_TASH.todense())

print(classification_report(np.argmax(one.fit_transform(Y_test_TASH.reshape(-1,1)), axis=1), np.argmax(predicted, axis=1)))

              precision    recall  f1-score   support

           0       0.35      0.53      0.42       238
           1       0.42      0.35      0.38       315
           2       0.47      0.37      0.41       284

    accuracy                           0.41       837
   macro avg       0.42      0.41      0.41       837
weighted avg       0.42      0.41      0.40       837



## KANSOAN

In [42]:
predicted = model.predict(X_test_KANSOAN.todense())

print(classification_report(np.argmax(one.fit_transform(np.asarray(Y_test_KANSOAN).reshape(-1,1)), axis=1), np.argmax(predicted, axis=1)))

              precision    recall  f1-score   support

           0       0.86      0.73      0.79      1352
           1       0.57      0.26      0.36      1416
           2       0.00      0.00      0.00         0

    accuracy                           0.49      2768
   macro avg       0.48      0.33      0.38      2768
weighted avg       0.71      0.49      0.57      2768



c:\users\ddayv\appdata\local\programs\python\python37\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


## LSTM T TWEET T TASH

In [43]:
vectorize_layer = TextVectorization(
 max_tokens=15000,
 output_mode='int',
 output_sequence_length=len(max(df_tweet['text'] )))

vocab = set_array(df_tweet['text'] )

vectorize_layer.adapt(np.unique(vocab))
len(vectorize_layer.get_vocabulary())


15000

In [44]:
text_KANSOAN

X_train3_, X_test_KANSOAN_, Y_train3_, Y_test_KANSOAN_ = train_test_split(text_KANSOAN, df['sentiment'], test_size=0.3)

In [45]:
model = tf.keras.Sequential([
    vectorize_layer,
    tf.keras.layers.Embedding(
        input_dim=len(vectorize_layer.get_vocabulary()),
        output_dim=64,mask_zero=True),
    
    tf.keras.layers.LSTM(50, activation='relu' ,return_sequences=True),
    tf.keras.layers.Dropout(0.3),
    
    tf.keras.layers.LSTM(25 , activation='tanh', return_sequences=True),
    tf.keras.layers.Dropout(0.3),
    
    tf.keras.layers.LSTM(10 , activation='tanh'),
    tf.keras.layers.Dropout(0.2),
    
    tf.keras.layers.Dense(3, activation='softmax')
])


model.compile(
    optimizer= tf.keras.optimizers.Adam(),
    loss=tf.keras.losses.categorical_crossentropy,
    metrics=['accuracy']
)

y_one = one.fit_transform(np.asarray(Y_train12).reshape(-1,1))
y_one_test = one.fit_transform(Y_test_TASH21.reshape(-1,1))

fit = model.fit(np.asarray(pre_X(X_train12)), y_one, epochs=10, batch_size=128 ,validation_data=(np.asarray(pre_X(X_test_TASH21)), y_one_test))

Epoch 1/10
64/64 [==============================] - 12s 190ms/step - loss: 1.0693 - accuracy: 0.4535 - val_loss: 1.1224 - val_accuracy: 0.3226
Epoch 2/10
64/64 [==============================] - 12s 192ms/step - loss: 0.8884 - accuracy: 0.5862 - val_loss: 1.1502 - val_accuracy: 0.4194
Epoch 3/10
64/64 [==============================] - 12s 183ms/step - loss: 0.6552 - accuracy: 0.7454 - val_loss: 1.4613 - val_accuracy: 0.3967
Epoch 4/10
64/64 [==============================] - 11s 171ms/step - loss: 0.4817 - accuracy: 0.8354 - val_loss: 1.6680 - val_accuracy: 0.4158
Epoch 5/10
64/64 [==============================] - 10s 163ms/step - loss: 0.3586 - accuracy: 0.8853 - val_loss: 1.9918 - val_accuracy: 0.4050
Epoch 6/10
64/64 [==============================] - 11s 165ms/step - loss: 0.2857 - accuracy: 0.9159 - val_loss: 2.1295 - val_accuracy: 0.4134
Epoch 7/10
64/64 [==============================] - 11s 170ms/step - loss: 0.2382 - accuracy: 0.9330 - val_loss: 2.2831 - val_accuracy: 0.4241

In [46]:
predicted = model.predict(np.asarray(pre_X(X_test_TASH21)))

print(classification_report(np.argmax(one.fit_transform(np.asarray(Y_test_TASH21).reshape(-1,1)), axis=1), np.argmax(predicted, axis=1)))

              precision    recall  f1-score   support

           0       0.43      0.40      0.41       258
           1       0.39      0.59      0.47       296
           2       0.46      0.25      0.32       283

    accuracy                           0.42       837
   macro avg       0.43      0.41      0.40       837
weighted avg       0.43      0.42      0.40       837



## UNILEX

In [47]:
predicted = model.predict(np.asarray(pre_X(X_test_text)))

print(classification_report(np.argmax(one.fit_transform(np.asarray(Y_test_text).reshape(-1,1)), axis=1), np.argmax(predicted, axis=1)))

              precision    recall  f1-score   support

           0       0.50      0.24      0.33      1271
           1       0.42      0.83      0.56      1434
           2       0.55      0.16      0.25      1083

    accuracy                           0.44      3788
   macro avg       0.49      0.41      0.38      3788
weighted avg       0.48      0.44      0.39      3788



## KANSOAN

In [48]:
predicted = model.predict(np.asarray(pre_X(X_test_KANSOAN1)))

print(classification_report(np.argmax(one.fit_transform(np.asarray(Y_test_KANSOAN1).reshape(-1,1)), axis=1), np.argmax(predicted, axis=1)))

              precision    recall  f1-score   support

           0       0.82      0.46      0.59      1341
           1       0.53      0.48      0.50      1427
           2       0.00      0.00      0.00         0

    accuracy                           0.47      2768
   macro avg       0.45      0.31      0.36      2768
weighted avg       0.67      0.47      0.55      2768



c:\users\ddayv\appdata\local\programs\python\python37\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


## CONV1D T TWEET T KANSOAN

In [49]:
model = tf.keras.Sequential([
    vectorize_layer,
    tf.keras.layers.Embedding(
        input_dim=len(vectorize_layer.get_vocabulary()),
        output_dim=64,
        mask_zero=True),
    
    tf.keras.layers.Conv1D(32,6, activation='relu'),
    tf.keras.layers.MaxPooling1D(2),
    tf.keras.layers.Flatten(),
    
    tf.keras.layers.Dense(16, activation='relu'),
    tf.keras.layers.Dense(3, activation='softmax')   
])

model.compile(
    optimizer= tf.keras.optimizers.Adam(),
    loss=tf.keras.losses.categorical_crossentropy,
    metrics=['accuracy']
)

y_one = one.fit_transform(np.asarray(Y_train12).reshape(-1,1))
y_one_test = one.fit_transform(Y_test_KANSOAN1.reshape(-1,1))

fit = model.fit(np.asarray(pre_X(X_train12)), y_one, epochs=10, batch_size=128)

Epoch 1/10
64/64 [==============================] - 2s 33ms/step - loss: 1.0649 - accuracy: 0.4468
Epoch 2/10
64/64 [==============================] - 2s 28ms/step - loss: 0.9403 - accuracy: 0.5596
Epoch 3/10
64/64 [==============================] - 2s 30ms/step - loss: 0.7008 - accuracy: 0.6705
Epoch 4/10
64/64 [==============================] - 2s 28ms/step - loss: 0.5148 - accuracy: 0.7630
Epoch 5/10
64/64 [==============================] - 2s 26ms/step - loss: 0.3834 - accuracy: 0.8515
Epoch 6/10
64/64 [==============================] - 2s 27ms/step - loss: 0.2922 - accuracy: 0.8937
Epoch 7/10
64/64 [==============================] - 2s 28ms/step - loss: 0.2053 - accuracy: 0.9320
Epoch 8/10
64/64 [==============================] - 2s 28ms/step - loss: 0.1462 - accuracy: 0.9568
Epoch 9/10
64/64 [==============================] - 2s 28ms/step - loss: 0.1061 - accuracy: 0.9683
Epoch 10/10
64/64 [==============================] - 2s 27ms/step - loss: 0.0824 - accuracy: 0.9746


In [50]:
predicted = model.predict(np.asarray(pre_X(X_test_KANSOAN1)))

print(classification_report(np.argmax(one.fit_transform(np.asarray(Y_test_KANSOAN1).reshape(-1,1)), axis=1), np.argmax(predicted, axis=1)))

              precision    recall  f1-score   support

           0       0.95      0.67      0.79      1341
           1       0.56      0.36      0.43      1427
           2       0.00      0.00      0.00         0

    accuracy                           0.51      2768
   macro avg       0.50      0.34      0.41      2768
weighted avg       0.75      0.51      0.60      2768



c:\users\ddayv\appdata\local\programs\python\python37\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


## UNILEX

In [51]:
predicted = model.predict(np.asarray(pre_X(X_test_text)))

print(classification_report(np.argmax(one.fit_transform(np.asarray(Y_test_text).reshape(-1,1)), axis=1), np.argmax(predicted, axis=1)))

              precision    recall  f1-score   support

           0       0.53      0.37      0.44      1271
           1       0.43      0.72      0.54      1434
           2       0.44      0.19      0.26      1083

    accuracy                           0.45      3788
   macro avg       0.47      0.43      0.41      3788
weighted avg       0.47      0.45      0.43      3788



## TASH

In [52]:
predicted = model.predict(np.asarray(pre_X(X_test_TASH21)))

print(classification_report(np.argmax(one.fit_transform(np.asarray(Y_test_TASH21).reshape(-1,1)), axis=1), np.argmax(predicted, axis=1)))

              precision    recall  f1-score   support

           0       0.40      0.32      0.35       258
           1       0.36      0.55      0.43       296
           2       0.46      0.29      0.36       283

    accuracy                           0.39       837
   macro avg       0.41      0.39      0.38       837
weighted avg       0.41      0.39      0.38       837



## BDR T TWEET T UNILEX

In [53]:
model = tf.keras.Sequential([
    vectorize_layer,
    tf.keras.layers.Embedding(
        input_dim=len(vectorize_layer.get_vocabulary()),
        output_dim=1,
        mask_zero=True),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(1)),

    tf.keras.layers.Dense(3, activation='softmax')
])

model.compile(
    optimizer= tf.keras.optimizers.Adam(),
    loss=tf.keras.losses.categorical_crossentropy,
    metrics=['accuracy']
)

y_one = one.fit_transform(np.asarray(Y_train12).reshape(-1,1))
y_one_test = one.fit_transform(np.asarray(Y_test_text).reshape(-1,1))

fit = model.fit(np.asarray(pre_X(X_train12)), y_one, epochs=10, batch_size=128 ,validation_data=(np.asarray(pre_X(X_test_text)), y_one_test))

Epoch 1/10
64/64 [==============================] - 4s 70ms/step - loss: 1.0884 - accuracy: 0.4487 - val_loss: 1.1134 - val_accuracy: 0.2859
Epoch 2/10
64/64 [==============================] - 3s 39ms/step - loss: 1.0715 - accuracy: 0.4505 - val_loss: 1.1319 - val_accuracy: 0.2859
Epoch 3/10
64/64 [==============================] - 2s 34ms/step - loss: 1.0587 - accuracy: 0.4505 - val_loss: 1.1412 - val_accuracy: 0.2859
Epoch 4/10
64/64 [==============================] - 2s 33ms/step - loss: 1.0399 - accuracy: 0.4510 - val_loss: 1.1371 - val_accuracy: 0.2859
Epoch 5/10
64/64 [==============================] - 2s 32ms/step - loss: 1.0067 - accuracy: 0.4985 - val_loss: 1.1120 - val_accuracy: 0.3131
Epoch 6/10
64/64 [==============================] - 2s 32ms/step - loss: 0.9532 - accuracy: 0.5568 - val_loss: 1.0905 - val_accuracy: 0.3807
Epoch 7/10
64/64 [==============================] - 2s 32ms/step - loss: 0.8870 - accuracy: 0.5995 - val_loss: 1.0912 - val_accuracy: 0.3749
Epoch 8/10
64

In [54]:
predicted = model.predict(np.asarray(pre_X(X_test_text)))

print(classification_report(np.argmax(one.fit_transform(np.asarray(Y_test_text).reshape(-1,1)), axis=1), np.argmax(predicted, axis=1)))

              precision    recall  f1-score   support

           0       0.36      0.96      0.52      1271
           1       0.00      0.00      0.00      1434
           2       0.53      0.18      0.27      1083

    accuracy                           0.37      3788
   macro avg       0.30      0.38      0.27      3788
weighted avg       0.27      0.37      0.25      3788



c:\users\ddayv\appdata\local\programs\python\python37\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


## TASH

In [55]:
predicted = model.predict(np.asarray(pre_X(X_test_TASH21)))

print(classification_report(np.argmax(one.fit_transform(np.asarray(Y_test_TASH21).reshape(-1,1)), axis=1), np.argmax(predicted, axis=1)))

              precision    recall  f1-score   support

           0       0.35      0.88      0.50       258
           1       0.00      0.00      0.00       296
           2       0.48      0.31      0.38       283

    accuracy                           0.38       837
   macro avg       0.27      0.40      0.29       837
weighted avg       0.27      0.38      0.28       837



c:\users\ddayv\appdata\local\programs\python\python37\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


## KANSOAN

In [56]:
predicted = model.predict(np.asarray(pre_X(X_test_KANSOAN1)))

print(classification_report(np.argmax(one.fit_transform(np.asarray(Y_test_KANSOAN1).reshape(-1,1)), axis=1), np.argmax(predicted, axis=1)))

              precision    recall  f1-score   support

           0       0.66      0.91      0.76      1341
           1       0.00      0.00      0.00      1427
           2       0.00      0.00      0.00         0

    accuracy                           0.44      2768
   macro avg       0.22      0.30      0.25      2768
weighted avg       0.32      0.44      0.37      2768



c:\users\ddayv\appdata\local\programs\python\python37\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\users\ddayv\appdata\local\programs\python\python37\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
